In [5]:
import sys
import os
import math
import random
import heapq 
import time
import copy
import gc
import numpy as np
import pandas as pd
from functools import reduce
from scipy.spatial.distance import pdist
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import faiss
import torch
import torch.nn as nn
import torch.nn.functional as F
print (torch.cuda.is_available())
print (torch.version.cuda)
print (torch.cuda.get_device_name(torch.cuda.current_device()))
torch.cuda.set_device(7)
print (torch.cuda.current_device())

True
9.0.176
GeForce RTX 2080 Ti
7


In [3]:
#1. Read data with List storage Name:[name],I:[img],Y[type]
root_dir = '/data/fjsdata/physionet/MIMIC-CXR' #the path of images
trainset = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/CBIR_train.csv" , sep=',')#load dataset
testset = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/CBIR_test.csv" , sep=',')#load dataset
tstart = time.time()
#read train image with CV
trN, trI, trY = [],[],[]
for iname, itype in np.array(trainset).tolist():
    try:
        image_path = os.path.join(root_dir, iname)
        img = cv2.resize(cv2.imread(image_path).astype(np.float32), (512, 512))#(1024,1024,3)->(512,512,3)
        trN.append(iname)
        trI.append(img)
        trY.append(itype)
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(trN),trainset.shape[0]))
    sys.stdout.flush()
print('The length of train set is %d'%len(trN))
#read test image with CV
teN, teI, teY = [],[],[]
for iname, itype in np.array(testset).tolist():
    try:
        image_path = os.path.join(root_dir, iname)
        img = cv2.resize(cv2.imread(image_path).astype(np.float32), (512, 512))#(1024,1024,3)->(512,512,3)
        teN.append(iname)
        teI.append(img)
        teY.append(itype)  
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(teN),testset.shape[0]))
    sys.stdout.flush()
print('The length of train set is %d'%len(teN))
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))

#Generate image pairs for model
def onlineGenImgPairs(spls=len(trY)):
    idx_sf = random.sample(range(0, len(trY)),spls)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trY[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trY[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf
trI1_sf, trI2_sf, trY_sf = onlineGenImgPairs()

9000 / 9000 The length of train set is 9000
900 / 900 The length of train set is 900
Completed buliding index in 1702 seconds


In [7]:
#define Hashing network with pytorch
class HNet(nn.Module): 
    def __init__(self,inChannels=3, outHashcode=16):
        super(HNet, self).__init__()
        #(channels, Height, Width)
        #layer1: Convolution, (3,512,512)->(8,256,256)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=8, kernel_size=3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.relu1 = nn.ReLU(inplace=True)
        #layer2: max pooling,(8,256,256)->(8,128,128)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(8)
        #layer: spatial attention, pass
        #layer3: Convolution, (8,128,128)->(2,64,64)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=2, kernel_size=3, padding=1, stride=2)
        self.bn3 = nn.BatchNorm2d(2)
        self.relu2 = nn.ReLU(inplace=True)
        #layer4: mean pooling, (2,64,64)->(2,32,32)
        self.avgpool = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn4 = nn.BatchNorm2d(2)
        #layer5: fully connected, 2*32*32->512
        self.fcl1 = nn.Linear(2*32*32,512)
        self.relu3 = nn.ReLU(inplace=True)
        #layer6: Hashing layer, 512->16
        self.fcl2 = nn.Linear(512,outHashcode)#
        self.tanh = nn.Tanh() #{-1,1}
              
    def forward(self,x):
        #input: (batch_size, in_channels, Height, Width)
        #output: (batch_size, out_channels, Height, Width)
        #layer1: convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #layer2: max pooling
        x = self.maxpool(x)
        x = self.bn2(x)
        #layer: attention,pass
        #layer3: Convolution
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.relu2(x)
        #layer4: mean pooling
        x = self.avgpool(x)
        x = self.bn4(x)
        #layer5:fully connected
        x = x.view(x.size(0),-1) #transfer three dims to one dim
        x = self.fcl1(x)
        x = self.relu3(x)
        #layer6: Hashing layer
        x = self.fcl2(x)
        x = self.tanh(x)
                
        return x
    
#https://pytorch-cn.readthedocs.io/zh/latest/    
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/layers/functional.py
class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
    
    def forward(self,h1,h2,y): 
        #h1=h2:NxD,y:N
        dim = h1.shape[1]
        euc_dist = F.pairwise_distance(h1, h2, p=2, eps=1e-06) # Calcualte Euclidean Distance
        sim_term = 0.5*(1-y)*euc_dist #penalize the similar iamge pairs when y=0
        unsim_term = 0.5*y*torch.clamp(self.margin*dim-euc_dist,0)#penalize the unsimlar image pairs when y =1
        reg_term = self.alpha * ( torch.sum((torch.abs(h1)-1),dim=1) + torch.sum((torch.abs(h2)-1),dim=1) ) #regularization term
        #loss = torch.mean(sim_term + unsim_term + reg_term) 
        loss = torch.sum(sim_term + unsim_term+ reg_term) 
        return loss

#define model
model = HNet(outHashcode=32).cuda()
criterion  = HashLossFunc(margin=0.3).cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
num_batches = len(trY_sf) // batchSize
for epoch in range(100):#iteration
    losses = []
    for i in range(num_batches):   
        optimizer.zero_grad() #grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY_sf), (i+1)*batchSize])
        I1_batch = torch.from_numpy(trI1_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        X1_batch = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(teI) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#train data with list: trData, trI, trF, trY
#test data with list: teData, teI, teF, teY
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = teY[i]
        map_item_score = {}
        for j, trVal in enumerate(trF):
            map_item_score[j] = pdist(np.vstack([teVal,trVal]),'hamming')
        ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in ranklist:
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 450 / 450 : loss = 20.753569Eopch:     1 mean_loss = 20.779539
 450 / 450 : loss = 22.686249Eopch:     2 mean_loss = 19.773121
 450 / 450 : loss = 25.414377Eopch:     3 mean_loss = 19.411851
 450 / 450 : loss = 23.817625Eopch:     4 mean_loss = 19.150364
 450 / 450 : loss = 22.570183Eopch:     5 mean_loss = 19.226598
 450 / 450 : loss = 22.139744Eopch:     6 mean_loss = 19.008948
 450 / 450 : loss = 22.422085Eopch:     7 mean_loss = 18.942513
 450 / 450 : loss = 21.859486Eopch:     8 mean_loss = 18.985912
 450 / 450 : loss = 23.426498Eopch:     9 mean_loss = 18.723805
 450 / 450 : loss = 22.714871Eopch:    10 mean_loss = 18.704162
 450 / 450 : loss = 23.939526Eopch:    11 mean_loss = 18.810273
 450 / 450 : loss = 22.518263Eopch:    12 mean_loss = 18.803040
 450 / 450 : loss = 22.079067Eopch:    13 mean_loss = 18.696478
 450 / 450 : loss = 22.759193Eopch:    14 mean_loss = 19.081631
 450 / 450 : loss = 21.495045Eopch:    15 mean_loss = 18.874965
 450 / 450 : loss = 25.023487Eopch:    1

In [ ]:
#define Attention-based Hashing network with pytorch
class SpatialAttention(nn.Module):#spatial attention layer
    def __init__(self):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    
class AHNet(nn.Module): #deep Hashint Network:DHNet
    def __init__(self,inChannels=3,outHashcode=16):
        super(AHNet, self).__init__()
        #(channels, Height, Width)
        #layer1: Convolution, (3,512,512)->(8,256,256)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=8, kernel_size=3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.relu1 = nn.ReLU(inplace=True)
        #layer2: max pooling,(8,256,256)->(8,128,128)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(8)
        #layer3: Spatial Attention Layer, (8,256,256)->(8,256,256)
        self.sa = SpatialAttention()
        #layer4: Convolution, (8,128,128)->(2,64,64)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=2, kernel_size=3, padding=1, stride=2)
        self.bn3 = nn.BatchNorm2d(2)
        self.relu2 = nn.ReLU(inplace=True)
        #layer5: mean pooling, (2,64,64)->(2,32,32)
        self.avgpool = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn4 = nn.BatchNorm2d(2)
        #layer6: fully connected, 2*32*32->512
        self.fcl1 = nn.Linear(2*32*32,512)
        self.relu3 = nn.ReLU(inplace=True)
        #layer7: Hashing layer, 512->16
        self.fcl2 = nn.Linear(512,outHashcode)#
        self.tanh = nn.Tanh() #{-1,1}
              
    def forward(self,x):
        #input: (batch_size, in_channels, Height, Width)
        #output: (batch_size, out_channels, Height, Width)
        #layer1: convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #layer2: max pooling
        x = self.maxpool(x)
        x = self.bn2(x)
        #layer3: Attention
        x = self.sa(x)*x
        #layer4: Convolution
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.relu2(x)
        #layer5: mean pooling
        x = self.avgpool(x)
        x = self.bn4(x)
        #layer6:fully connected
        x = x.view(x.size(0),-1) #transfer three dims to one dim
        x = self.fcl1(x)
        x = self.relu3(x)
        #layer7: Hashing layer
        x = self.fcl2(x)
        x = self.tanh(x)
                
        return x
    
#https://pytorch-cn.readthedocs.io/zh/latest/    
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/layers/functional.py
class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
    
    def forward(self,h1,h2,y): 
        #h1=h2:NxD,y:N
        dim = h1.shape[1]
        euc_dist = F.pairwise_distance(h1, h2, p=2, eps=1e-06) # Calcualte Euclidean Distance
        sim_term = 0.5*(1-y)*euc_dist #penalize the similar iamge pairs when y=0
        unsim_term = 0.5*y*torch.clamp(self.margin*dim-euc_dist,0)#penalize the unsimlar image pairs when y =1
        reg_term = self.alpha * ( torch.sum((torch.abs(h1)-1),dim=1) + torch.sum((torch.abs(h2)-1),dim=1) ) #regularization term
        #loss = torch.mean(sim_term + unsim_term + reg_term) 
        loss = torch.sum(sim_term + unsim_term+ reg_term) 
        return loss
    
#define model
model = AHNet(outHashcode=32).cuda()
criterion  = HashLossFunc(margin=0.3).cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
num_batches = len(trY_sf) // batchSize
for epoch in range(100):#iteration
    losses = []
    for i in range(num_batches):   
        optimizer.zero_grad() #grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY_sf), (i+1)*batchSize])
        I1_batch = torch.from_numpy(trI1_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        X1_batch = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(teI) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#train data with list: trData, trI, trF, trY
#test data with list: teData, teI, teF, teY
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = teY[i]
        map_item_score = {}
        for j, trVal in enumerate(trF):
            map_item_score[j] = pdist(np.vstack([teVal,trVal]),'hamming')
        ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in ranklist:
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 450 / 450 : loss = 27.219505Eopch:     1 mean_loss = 21.167239
 450 / 450 : loss = 24.781539Eopch:     2 mean_loss = 20.102185
 450 / 450 : loss = 24.754116Eopch:     3 mean_loss = 19.836653
 450 / 450 : loss = 25.758965Eopch:     4 mean_loss = 19.797267
 450 / 450 : loss = 26.395975Eopch:     5 mean_loss = 19.706234
 450 / 450 : loss = 27.093191Eopch:     6 mean_loss = 19.662256
 450 / 450 : loss = 24.192425Eopch:     7 mean_loss = 19.812892
 450 / 450 : loss = 23.147552Eopch:     9 mean_loss = 21.167607
 450 / 450 : loss = 23.290432Eopch:    10 mean_loss = 20.121827
 450 / 450 : loss = 23.735516Eopch:    11 mean_loss = 19.788363
 450 / 450 : loss = 23.262829Eopch:    12 mean_loss = 19.568516
 450 / 450 : loss = 23.717112Eopch:    13 mean_loss = 19.641504
 450 / 450 : loss = 22.541138Eopch:    14 mean_loss = 19.323304
 450 / 450 : loss = 22.510509Eopch:    15 mean_loss = 19.508521
 450 / 450 : loss = 22.337872Eopch:    16 mean_loss = 19.511593
 450 / 450 : loss = 21.815596Eopch:    1

In [26]:
#generate dataset
dataset = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/train.csv" , sep=',')
#print (dataset.columns)
#print (dataset['view'].value_counts())
dataset = dataset[dataset['view']=='frontal']#only get frontal view
dataset = dataset[['path','Pneumonia']]
#print (dataset['Pneumonia'].value_counts())
trainset, testset = [], []
ds =dataset[dataset['Pneumonia']==0.0].sample(3300).sample(frac=1)#26296-3300, negative
testset.extend(np.array(ds).tolist()[0:300])
trainset.extend(np.array(ds).tolist()[300:])
ds =dataset[dataset['Pneumonia']==1.0].sample(3300).sample(frac=1)#18327-3300, positive
testset.extend(np.array(ds).tolist()[0:300])
trainset.extend(np.array(ds).tolist()[300:])
ds =dataset[dataset['Pneumonia']==-1.0].sample(3300).sample(frac=1)#19681-3300, positive
testset.extend(np.array(ds).tolist()[0:300])
trainset.extend(np.array(ds).tolist()[300:])
print (len(trainset))
print (len(testset))
pd.DataFrame(trainset).to_csv('/data/fjsdata/physionet/MIMIC-CXR/CBIR_train.csv',index=False)
pd.DataFrame(testset).to_csv('/data/fjsdata/physionet/MIMIC-CXR/CBIR_test.csv',index=False)
del ds,dataset
gc.collect() #release cpu memory

9000
900


7